# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 程式區塊 A
# 將需要的都import進來
import os
import copy
import time
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import math
import warnings
from scipy                   import stats
from sklearn.preprocessing   import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model    import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score
# 將較長的函式改名一下
MME  = MinMaxScaler()
LR   = LogisticRegression()
LIR  = LinearRegression()
LE   = LabelEncoder()
PDDF = pd.DataFrame()
# 一些必要的設定
warnings.filterwarnings('ignore')
%matplotlib inline

# 設定【data的資料夾路徑】，命名為【data_folder】
data_folder = 'C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data'

In [2]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_train = os.path.join(data_folder, 'titanic_train.csv')
t002_train = pd.read_csv(t001_train)
print('Path of read in data: %s' %t001_train)
print(t002_train.shape)
t002_train.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_train.csv
(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_test  = os.path.join(data_folder,  'titanic_test.csv')
t002_test  = pd.read_csv(t001_test)
print('Path of read in data: %s' %t001_test)
print(t002_test.shape)
t002_test.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_test.csv
(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# 程式區塊 B-1：train取【Survived】為Y、test取【PassengerId】為最終將比對的唯一識別
train_Y     = t002_train['Survived']
test_unique = t002_test['PassengerId']
# 程式區塊 B-2：train捨棄【PassengerId,Survived】、test捨棄【PassengerId】。
t003_train = t002_train.drop(['PassengerId', 'Survived'] , axis=1)
t003_test  = t002_test.drop(['PassengerId'] , axis=1)
print(t003_train.shape)
print(t003_test.shape)
# 程式區塊 B-3：設計t005，把train和test合併，UNION ALL的概念。
t004 = pd.concat([t003_train,t003_test])
print(t004.shape)

(891, 10)
(418, 10)
(1309, 10)


In [5]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for a, b in zip(t004.dtypes, t004.columns):
    if a == 'object':
        num_features.append(b)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



In [6]:
# 削減文字型欄位, 只剩數值型欄位
t005 = t004[num_features]
# 將空值補0
t006 = t005.fillna('None')
# 計算訓練資料的筆數
train_cnt = train_Y.shape[0]
print(train_cnt)
t006.head(5)

891


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [7]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
for d in t006.columns:
    PDDF[d] = LE.fit_transform(t006[d])
train_X = PDDF[:train_cnt]
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.780004837244799
time : 0.07598757743835449 sec


In [8]:
# 如果欄位類別數太多, 需要先剔除, 否則會 overfit (正確率會到達 1.0)
# 意思是，例如唯一識別欄位，如果拿來做這件事，則每個唯一識別內容等於專門拿來辨識唯一個人，那當然可以100%正確，但這並沒有意義。
train_X.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
dtype: int64

那該如何處理呢？按照教材的意思，會分別有以下幾個動作：

1. 把【train_Y】join回正在處理中的【train_X】(這時候只是t006[:train_cnt]而已)，成為t007。  
2. 對每個t006的欄位，逐一group by之後，對【train_Y】，也就是【Survived】取mean，成為t008。  
3. t008針對取mean完的欄位重新命名，此時t008和t007有相同的欄位數(只是命名不同)。  
4. 然後把t008所有欄位的mean數值，join回去t007，但t007和t008的資料筆數可能不同啊(因為group by過)，所以要用pd.merge，【on=d】的意思是以欄位做比對，【how='left'】就是left join。
5. 接著drop掉t007原本的欄位(取mean之前的欄位)。
6. 接著就能看出mean的數值內容是不是因為種類太多，而只剩下1或0兩種辨識，於是知道該把【'Name_mean', 'Ticket_mean'】給drop掉。但令人不解的就是，上面【train_X.nunique()】時，不就看得出能把【'Name', 'Ticket'】給drop掉了嗎？做1~4的目的到底是什麼？為什麼不是先drop完【'Name', 'Ticket'】，再來做這件事呢？  

可發現2~4是每個欄位做完在換下一個欄位，把所有欄位跑完，即迴圈。

In [9]:
# 1. 把【train_Y】join回正在處理中的【train_X】(這時候只是t006[:train_cnt]而已)，成為t007。
t007 = pd.concat([t006[:train_cnt], train_Y], axis=1)
print(t007.shape)
t007.head(10)

(891, 6)


,Name,Sex,Ticket,Cabin,Embarked,Survived
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1
4,"Allen, Mr. William Henry",male,373450,None,S,0
5,"Moran, Mr. James",male,330877,None,Q,0
6,"McCarthy, Mr. Timothy J",male,17463,E46,S,0
7,"Palsson, Master. Gosta Leonard",male,349909,None,S,0
8,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,347742,None,S,1
9,"Nasser, Mrs. Nicholas (Adele Achem)",female,237736,None,C,1


In [10]:
# 2. 對每個t006的欄位，逐一group by之後，對【train_Y】，也就是【Survived】取mean，成為t008。
# 此為測試內容，幫助了解等下迴圈內到底做了什麼事，可嘗試替換成【'Name','Sex','Cabin'】
t008 = t007.groupby(['Cabin'])['Survived'].mean().reset_index()
t008

,Cabin,Survived
0,A10,0.000000
1,A14,0.000000
2,A16,1.000000
3,A19,0.000000
4,A20,1.000000
5,A23,1.000000
6,A24,0.000000
7,A26,1.000000
8,A31,1.000000
9,A32,0.000000


In [11]:
for d in t006.columns:
    # 2. 對每個t006的欄位，逐一group by之後，對【train_Y】，也就是【Survived】取mean，成為t008。
    t008 = t007.groupby([d])['Survived'].mean().reset_index()
    # 3. t008針對取mean完的欄位重新命名，此時t008和t007有相同的欄位數(只是命名不同)。
    t008.columns = [d, f'{d}_mean']
    # 4. 然後把t008所有欄位的mean數值，join回去t007，但t007和t008的資料筆數可能不同啊(因為group by過)，所以要用pd.merge。 
    t007 = pd.merge(t007, t008, on=d, how='left')
    # 5. 接著drop掉t007原本的欄位(取mean之前的欄位)。
    t007 = t007.drop([d] , axis=1)

In [12]:
print(t007.shape)
t007.head(10)

(891, 6)


,Survived,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0,0.188908,0.0,0.299854,0.336957
1,1,1,0.742038,1.0,1.000000,0.553571
2,1,1,0.742038,1.0,0.299854,0.336957
3,1,1,0.742038,0.5,0.500000,0.336957
4,0,0,0.188908,0.0,0.299854,0.336957
5,0,0,0.188908,0.0,0.299854,0.389610
6,0,0,0.188908,0.0,0.000000,0.336957
7,0,0,0.188908,0.0,0.299854,0.336957
8,1,1,0.742038,1.0,0.299854,0.336957
9,1,1,0.742038,0.5,0.299854,0.553571


In [13]:
# 6. 接著就能看出mean的數值內容是不是因為種類太多，而只剩下1或0兩種辨識，於是知道該把【'Name_mean', 'Ticket_mean'】給drop掉。
# 但令人不解的就是，上面【train_X.nunique()】時，不就看得出能把【'Name', 'Ticket'】給drop掉了嗎？
# 做1~4的目的到底是什麼？為什麼不是先drop完【'Name', 'Ticket'】，再來做這件事呢？
t009 = t007.drop(['Survived', 'Name_mean', 'Ticket_mean'] , axis=1)
print(t009.shape)
t009.head(10)

(891, 3)


,Sex_mean,Cabin_mean,Embarked_mean
0,0.188908,0.299854,0.336957
1,0.742038,1.000000,0.553571
2,0.742038,0.299854,0.336957
3,0.742038,0.500000,0.336957
4,0.188908,0.299854,0.336957
5,0.188908,0.299854,0.389610
6,0.188908,0.000000,0.336957
7,0.188908,0.299854,0.336957
8,0.742038,0.299854,0.336957
9,0.742038,0.299854,0.553571


In [14]:
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, t009, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.8350366889413987
time : 0.03399229049682617 sec


## -------------分隔線-------------：試試先drop掉不要的欄位會怎樣

In [15]:
t006 = t006.drop(['Name','Ticket'] , axis=1)

In [16]:
t007 = pd.concat([t006[:train_cnt], train_Y], axis=1)

In [17]:
for d in t006.columns:
    # 2. 對每個t006的欄位，逐一group by之後，對【train_Y】，也就是【Survived】取mean，成為t008。
    t008 = t007.groupby([d])['Survived'].mean().reset_index()
    # 3. t008針對取mean完的欄位重新命名，此時t008和t007有相同的欄位數(只是命名不同)。
    t008.columns = [d, f'{d}_mean']
    # 4. 然後把t008所有欄位的mean數值，join回去t007，但t007和t008的資料筆數可能不同啊(因為group by過)，所以要用pd.merge。 
    t007 = pd.merge(t007, t008, on=d, how='left')
    # 5. 接著drop掉t007原本的欄位(取mean之前的欄位)。
    t007 = t007.drop([d] , axis=1)

In [18]:
print(t007.shape)
t007.head(10)

(891, 4)


,Survived,Sex_mean,Cabin_mean,Embarked_mean
0,0,0.188908,0.299854,0.336957
1,1,0.742038,1.000000,0.553571
2,1,0.742038,0.299854,0.336957
3,1,0.742038,0.500000,0.336957
4,0,0.188908,0.299854,0.336957
5,0,0.188908,0.299854,0.389610
6,0,0.188908,0.000000,0.336957
7,0,0.188908,0.299854,0.336957
8,1,0.742038,0.299854,0.336957
9,1,0.742038,0.299854,0.553571


In [19]:
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, t009, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.8350366889413987
time : 0.051993370056152344 sec


呃...對...一樣...那照理講看完【train_X.nunique()】就要drop了。

### Day23教材方向和目標
1. [drop時機點的問題](https://www.cupoy.com/qa/kwassist/ai_tw/0000016ABA2AD1160000009A6375706F795F72656C656173655155455354)
2. pd.concat：預設值(不寫時)【axis=0】是UNION ALL的概念，寫出【axis=0】時，是以index做join的概念。
3. for d in t006.columns:做的那四件事情，可以學一下。
4. 終於看到【left join on col=col】了！  
   t007 = pd.merge(t007, t008, on=d, how='left')  
   只是如果欄位名稱不同，該怎辦啊？
5. 如何把欄位命名以變數的方式製作。
   t008.columns = [d, f'{d}_mean']

### Day23忽略部分
1. 如果欄位名稱不同，該怎麼join？

### Day23其他補充
for迴圈內到底做了什麼事情，也不講清楚...  
而且跑完，還是得人工指定要drop的欄位？